In [2]:
# Importing required Packages
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import *
from scipy.integrate import *

In [3]:
# Changing directory to Waveform location
%cd C:\Users\trent\Desktop\ML Paper -02 Data\Vivek_2022\CF_25_CCS_0-6

C:\Users\trent\Desktop\ML Paper -02 Data\Vivek_2022\CF_25_CCS_0-6


In [6]:
# Reading calibration data to set boundaries on graphical interface
calibration = np.loadtxt("Filtered Waveform_Pressure_SampleCalibration.txt")
# Splitting columns into time and amplitude
time_cal = calibration[:,0]
amplitude_cal = calibration[:,1]

In [7]:
# Setting window index's

cal_w1 = 1061
cal_w2 = 1163
samp_w1 = 1300
samp_w2 = 2000
proc_W1 = 111
proc_W2 = 331

In [8]:
# Filtered Waveform Processing

# Creating finalized metric of each filtered waveform
wf_metric = np.zeros(35)
plt.figure(figsize=(12,10))
# creating storage for normalized fast fourier transform (FFT) outputs
norm_FFT = np.zeros((2251, 35))

<Figure size 1200x1000 with 0 Axes>

In [9]:
# Loading all filtered samples into previously allocated metrics
for i in range(35):
    # changing waveform number in according to loop
    wf_1 = np.loadtxt(f'Filtered Waveform_Pressure_Sample{i+1}.txt')
    # cropping sample and calibration waveforms
    y_cal_filt = calibration[:,i]
    y_samp_filt = wf_1[:,1]
    y_filt_s1 = y_cal_filt[cal_w1:cal_w2]
    y_filt_s2 = y_samp_filt[samp_w1:samp_w2]
    # Applying FFT to analyze freq domain by taking absolute value for amplitude spectrum
    ampspec_1 = np.abs(fft(y_filt_s1), 11000)
    ampspec_2 = np.abs(fft(y_filt_s2), 11000)
